## Общие данные по Oracle XML DB
Погрузимся в детали Oracle XML DB! Это мощная технология, которая интегрирует XML в реляционную СУБД Oracle.

Архитектура Oracle XML DB

Способы хранения XMLType

```sql
-- 1. CLOB хранение (неструктурированное)
CREATE TABLE students_clob (
    id NUMBER PRIMARY KEY,
    student_data XMLTYPE
) XMLTYPE student_data STORE AS SECUREFILE CLOB;

-- 2. Object-Relational хранение (структурированное)
CREATE TABLE students_structured (
    id NUMBER PRIMARY KEY,
    student_data XMLTYPE
) XMLTYPE student_data STORE AS OBJECT RELATIONAL
  XMLSCHEMA "http://university.edu/student.xsd"
  ELEMENT "student";

-- 3. Binary XML (рекомендуется)
CREATE TABLE students_binary (
    id NUMBER PRIMARY KEY,
    student_data XMLTYPE
) XMLTYPE student_data STORE AS SECUREFILE BINARY XML;
```

Реализация отношений "многие ко многим"

Подход 1: Встраивание коллекций в XML

```sql
-- Создаем типы для коллекций
CREATE TYPE course_enrollment_typ AS OBJECT (
    course_code VARCHAR2(20),
    course_title VARCHAR2(100),
    enrolled_date DATE,
    grade VARCHAR2(2),
    credits NUMBER
);

CREATE TYPE course_enrollments_typ AS TABLE OF course_enrollment_typ;

-- Таблица студентов с встроенными курсами
CREATE TABLE students (
    student_id NUMBER PRIMARY KEY,
    student_data XMLTYPE
);

-- Вставка данных со встроенными курсами
INSERT INTO students VALUES (
    123,
    XMLTYPE('
    <student id="123">
        <personal>
            <name>John Doe</name>
            <email>john@university.edu</email>
            <birthdate>2000-05-15</birthdate>
        </personal>
        <enrollments>
            <course code="MATH101">
                <title>Calculus I</title>
                <enrolled>2024-01-15</enrolled>
                <grade>A</grade>
                <credits>5</credits>
            </course>
            <course code="CS101">
                <title>Programming Basics</title>
                <enrolled>2024-01-20</enrolled>
                <grade>B+</grade>
                <credits>6</credits>
            </course>
        </enrollments>
    </student>
    ')
);
```

Подход 2: Раздельные XML документы + ссылки

```sql
-- Таблица студентов
CREATE TABLE students_xml (
    student_id NUMBER PRIMARY KEY,
    student_data XMLTYPE
);

-- Таблица курсов  
CREATE TABLE courses_xml (
    course_code VARCHAR2(20) PRIMARY KEY,
    course_data XMLTYPE
);

-- Таблица связей многие-ко-многим
CREATE TABLE enrollments_xml (
    enrollment_id NUMBER PRIMARY KEY,
    enrollment_data XMLTYPE
);

-- Вставка студентов
INSERT INTO students_xml VALUES (
    123,
    XMLTYPE('
    <student>
        <studentId>123</studentId>
        <name>John Doe</name>
        <email>john@university.edu</email>
    </student>
    ')
);

-- Вставка курсов
INSERT INTO courses_xml VALUES (
    'MATH101',
    XMLTYPE('
    <course>
        <courseCode>MATH101</courseCode>
        <title>Calculus I</title>
        <credits>5</credits>
        <instructor>Dr. Smith</instructor>
    </course>
    ')
);

-- Вставка связей
INSERT INTO enrollments_xml VALUES (
    1,
    XMLTYPE('
    <enrollment>
        <enrollmentId>1</enrollmentId>
        <studentRef>123</studentRef>
        <courseRef>MATH101</courseRef>
        <enrollmentDate>2024-01-15</enrollmentDate>
        <grade>A</grade>
        <status>ACTIVE</status>
    </enrollment>
    ')
);
```

Особенности Oracle XML DB

1. XML Indexing

```sql
-- Создание индексов для производительности
-- Контекстный индекс для полнотекстового поиска
CREATE INDEX student_name_ctx_idx ON students_xml (
    extract(student_data, '/student/name/text()').getStringVal()
);

-- Функциональный индекс для часто используемых путей
CREATE INDEX student_email_idx ON students_xml (
    extractValue(student_data, '/student/email')
);

-- XMLIndex для произвольных XPath запросов
CREATE XMLINDEX student_xml_idx ON students_xml (student_data)
INDEXTYPE IS XDB.XMLIndex;

-- Создание вторичного индекса на конкретные пути
ALTER INDEX student_xml_idx
ADD PARAMETERS ('PATHS (//email AS VARCHAR2(100))');
```

2. XML Schema Validation

```sql
-- Регистрация XML Schema
BEGIN
    DBMS_XMLSCHEMA.registerSchema(
        'student-enrollment.xsd',
        '<xsd:schema xmlns:xsd="http://www.w3.org/2001/XMLSchema"
                    targetNamespace="http://university.edu/student">
            <xsd:element name="student">
                <xsd:complexType>
                    <xsd:sequence>
                        <xsd:element name="studentId" type="xsd:integer"/>
                        <xsd:element name="name" type="xsd:string"/>
                        <xsd:element name="email" type="xsd:string"/>
                        <xsd:element name="enrollments" minOccurs="0">
                            <xsd:complexType>
                                <xsd:sequence>
                                    <xsd:element name="course" maxOccurs="unbounded">
                                        <xsd:complexType>
                                            <xsd:sequence>
                                                <xsd:element name="courseCode" type="xsd:string"/>
                                                <xsd:element name="title" type="xsd:string"/>
                                                <xsd:element name="grade" type="xsd:string"/>
                                            </xsd:sequence>
                                        </xsd:complexType>
                                    </xsd:element>
                                </xsd:sequence>
                            </xsd:complexType>
                        </xsd:element>
                    </xsd:sequence>
                </xsd:complexType>
            </xsd:element>
        </xsd:schema>',
        TRUE, TRUE, FALSE
    );
END;
/

-- Таблица с валидацией по схеме
CREATE TABLE students_validated (
    student_data XMLTYPE
) XMLTYPE student_data STORE AS BINARY XML
XMLSCHEMA "http://university.edu/student" ELEMENT "student";
```

Отличия SQL запросов в Oracle XML DB

Обычные SQL запросы vs XML-ориентированные

Традиционный SQL (реляционный):

```sql
-- JOIN трех таблиц
SELECT 
    s.student_name,
    c.course_title,
    e.grade,
    e.enrollment_date
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
JOIN courses c ON e.course_id = c.course_id
WHERE c.course_code = 'MATH101'
  AND e.grade = 'A';
```

Oracle XML DB SQL запросы:

Вариант 1: EXTRACT и EXTRACTVALUE (устаревшие)

```sql
-- Простые извлечения (устаревший подход)
SELECT 
    extractValue(student_data, '/student/name') as student_name,
    extractValue(student_data, '/student/email') as email
FROM students_xml
WHERE extractValue(student_data, '/student/enrollments/course/grade') = 'A';
```

Вариант 2: XMLTABLE (рекомендуется)

```sql
-- Преобразование XML в реляционный формат
SELECT 
    s.student_id,
    x.name,
    x.email,
    c.course_code,
    c.grade
FROM students_xml s,
XMLTable(
    '/student' PASSING s.student_data
    COLUMNS
        name VARCHAR2(100) PATH 'name',
        email VARCHAR2(100) PATH 'email',
        courses XMLType PATH 'enrollments'
) x,
XMLTable(
    '/enrollments/course' PASSING x.courses
    COLUMNS
        course_code VARCHAR2(20) PATH 'courseCode',
        grade VARCHAR2(2) PATH 'grade'
) c
WHERE c.grade = 'A';
```

Вариант 3: XMLQuery

```sql
-- Использование XQuery в SQL
SELECT 
    student_id,
    XMLQuery(
        '/student/name/text()' PASSING student_data RETURNING CONTENT
    ).getStringVal() as student_name,
    XMLQuery(
        'count(/student/enrollments/course)' PASSING student_data RETURNING CONTENT
    ).getNumberVal() as course_count
FROM students_xml
WHERE XMLEXISTS(
    '/student[enrollments/course/grade = "A"]' PASSING student_data
);
```

Сложные запросы с агрегациями

Традиционный SQL:

```sql
-- Статистика по курсам
SELECT 
    c.course_code,
    c.course_title,
    COUNT(e.student_id) as student_count,
    AVG(CASE 
        WHEN e.grade = 'A' THEN 4
        WHEN e.grade = 'B' THEN 3 
        WHEN e.grade = 'C' THEN 2
        ELSE 1 
    END) as avg_grade_points
FROM courses c
LEFT JOIN enrollments e ON c.course_code = e.course_code
GROUP BY c.course_code, c.course_title;
```

Oracle XML DB:

```sql
-- Статистика из XML данных
WITH course_stats AS (
    SELECT 
        c.course_code,
        x.course_title,
        COUNT(s.student_id) as student_count,
        AVG(CASE 
            WHEN y.grade = 'A' THEN 4
            WHEN y.grade = 'B' THEN 3 
            WHEN y.grade = 'C' THEN 2
            ELSE 1 
        END) as avg_grade_points
    FROM courses_xml c,
    XMLTable(
        '/course' PASSING c.course_data
        COLUMNS
            course_title VARCHAR2(100) PATH 'title'
    ) x,
    enrollments_xml e,
    XMLTable(
        '/enrollment' PASSING e.enrollment_data
        COLUMNS
            student_ref NUMBER PATH 'studentRef',
            course_ref VARCHAR2(20) PATH 'courseRef',
            grade VARCHAR2(2) PATH 'grade'
    ) y,
    students_xml s
    WHERE c.course_code = y.course_ref
      AND s.student_id = y.student_ref
    GROUP BY c.course_code, x.course_title
)
SELECT * FROM course_stats ORDER BY avg_grade_points DESC;
```

Особенности производительности

1. План запросов с XMLIndex

```sql
-- Анализ плана выполнения
EXPLAIN PLAN FOR
SELECT /*+ INDEX(student_xml_idx) */ 
    extractValue(student_data, '/student/name') as name
FROM students_xml
WHERE XMLEXISTS('/student[email="john@university.edu"]' PASSING student_data);

SELECT * FROM TABLE(DBMS_XPLAN.DISPLAY);
```

2. Оптимизация через материализованные представления

```sql
-- Создание материализованного представления для частых запросов
CREATE MATERIALIZED VIEW student_courses_mv
BUILD IMMEDIATE
REFRESH COMPLETE ON DEMAND
AS
SELECT 
    s.student_id,
    extractValue(s.student_data, '/student/name') as student_name,
    extractValue(c.course_data, '/course/title') as course_title,
    extractValue(e.enrollment_data, '/enrollment/grade') as grade
FROM students_xml s, courses_xml c, enrollments_xml e
WHERE extractValue(e.enrollment_data, '/enrollment/studentRef') = s.student_id
  AND extractValue(e.enrollment_data, '/enrollment/courseRef') = c.course_code;
```

Практические примеры использования

Пример 1: Поиск и фильтрация

```sql
-- Найти всех студентов с определенной оценкой по курсу
SELECT 
    s.student_id,
    XMLQuery('/student/name/text()' PASSING s.student_data RETURNING CONTENT).getStringVal() as name,
    course_data.course_code,
    course_data.grade
FROM students_xml s,
LATERAL(
    SELECT 
        extractValue(column_value, 'courseCode') as course_code,
        extractValue(column_value, 'grade') as grade
    FROM TABLE(XMLSequence(extract(s.student_data, '/student/enrollments/course')))
) course_data
WHERE course_data.grade = 'A'
  AND course_data.course_code = 'MATH101';
```

Пример 2: Обновление XML данных

```sql
-- Обновление оценки студента
UPDATE students_xml 
SET student_data = updateXML(
    student_data,
    '/student/enrollments/course[courseCode="MATH101"]/grade/text()',
    'A+'
)
WHERE student_id = 123;

-- Более эффективное обновление с существованием проверки
UPDATE students_xml 
SET student_data = XMLQuery(
    'copy $i := $s modify (
        for $j in $i/student/enrollments/course[courseCode="MATH101"]/grade
        return replace value of node $j with "A+"
    ) return $i'
    PASSING student_data AS "s" RETURNING CONTENT
)
WHERE XMLEXISTS(
    '/student/enrollments/course[courseCode="MATH101"]' 
    PASSING student_data
);
```

Ключевые преимущества Oracle XML DB

1. Интеграция - полная интеграция XML и реляционных данных

2. Производительность - специализированные индексы и хранилища

3. Стандарты - поддержка XQuery, XPath, XML Schema

4. Безопасность - ролевой доступ к XML документам

5. Versioning - встроенное управление версиями XML

Ограничения и рекомендации

Когда использовать:

· ✅ Сложные иерархические данные
· ✅ Гибкая, часто меняющаяся схема
· ✅ Интеграция с XML-ориентированными системами
· ✅ Документо-ориентированные приложения

Когда не использовать:

· ❌ Простые реляционные структуры
· ❌ Высоконагруженные OLTP системы
· ❌ Команда без опыта работы с XML
· ❌ Требуется простая миграция данных

Oracle XML DB предоставляет мощный гибридный подход, сочетающий гибкость XML с производительностью и надежностью реляционной СУБД Oracle.


## XQUERY в SQL
1. Базовые функции XQuery в SQL

EXTRACTVALUE - извлечение скалярных значений

```sql
-- Простейшее извлечение значения из XML
SELECT 
    student_id,
    EXTRACTVALUE(student_data, '/student/name') as student_name,
    EXTRACTVALUE(student_data, '/student/email') as email
FROM students_xml
WHERE EXTRACTVALUE(student_data, '/student/status') = 'ACTIVE';
```

Что происходит:

· EXTRACTVALUE извлекает текстовое значение по XPath
· Возвращает VARCHAR2, NUMBER или DATE
· Устарел в новых версиях Oracle

EXISTSNODE - проверка существования элемента

```sql
-- Проверка наличия элементов в XML
SELECT student_id
FROM students_xml
WHERE EXISTSNODE(student_data, '/student/enrollments/course[grade="A"]') = 1;
```

Что происходит:

· Возвращает 1 если элемент существует, 0 если нет
· Используется для фильтрации

2. Функции XMLQUERY и XMLEXISTS

Базовое использование XMLQUERY

```sql
-- Извлечение данных с XMLQUERY
SELECT 
    student_id,
    XMLQUERY(
        '/student/name/text()' 
        PASSING student_data 
        RETURNING CONTENT
    ).getStringVal() as student_name,
    XMLQUERY(
        '/student/enrollments/count(course)' 
        PASSING student_data 
        RETURNING CONTENT
    ).getNumberVal() as course_count
FROM students_xml;
```

Фильтрация с XMLEXISTS

```sql
-- Фильтрация с использованием XPath предикатов
SELECT 
    student_id,
    XMLQUERY('/student/name/text()' PASSING student_data RETURNING CONTENT).getStringVal() as name
FROM students_xml
WHERE XMLEXISTS(
    '/student[enrollments/course/grade="A" and personal/age > 20]' 
    PASSING student_data
);
```

3. XMLTABLE - преобразование XML в реляционный формат

Простое преобразование

```sql
-- Базовое использование XMLTable
SELECT 
    s.student_id,
    x.student_name,
    x.email,
    x.registration_date
FROM students_xml s,
XMLTable(
    '/student' 
    PASSING s.student_data
    COLUMNS
        student_name VARCHAR2(100) PATH 'name',
        email        VARCHAR2(100) PATH 'email',
        registration_date DATE PATH 'registrationDate'
) x;
```

Работа с атрибутами

```sql
-- Извлечение атрибутов и элементов
SELECT 
    x.student_id,
    x.student_name,
    x.faculty
FROM students_xml s,
XMLTable(
    '/student' 
    PASSING s.student_data
    COLUMNS
        student_id   NUMBER       PATH '@id',        -- Атрибут id
        student_name VARCHAR2(100) PATH 'name',      -- Элемент name
        faculty      VARCHAR2(50)  PATH '@faculty'   -- Атрибут faculty
) x;
```

4. Обработка коллекций и повторяющихся элементов

Разворачивание повторяющихся элементов

```sql
-- Каждый курс становится отдельной строкой
SELECT 
    s.student_id,
    x.student_name,
    c.course_code,
    c.course_title,
    c.grade
FROM students_xml s,
XMLTable(
    '/student' 
    PASSING s.student_data
    COLUMNS
        student_name VARCHAR2(100) PATH 'name',
        courses      XMLType       PATH 'enrollments'
) x,
XMLTable(
    '/enrollments/course' 
    PASSING x.courses
    COLUMNS
        course_code  VARCHAR2(20) PATH 'courseCode',
        course_title VARCHAR2(100) PATH 'title',
        grade        VARCHAR2(2)  PATH 'grade'
) c;
```

5. Условная логика в XQuery

Использование FLWOR выражений

```sql
-- Сложная логика с FLWOR (For-Let-Where-Order-Return)
SELECT 
    student_id,
    XMLQuery(
        'for $course in /student/enrollments/course
         where $course/grade = "A"
         order by $course/title
         return <excellent_course>{$course/title}</excellent_course>' 
        PASSING student_data 
        RETURNING CONTENT
    ).getStringVal() as excellent_courses
FROM students_xml;
```

Условное преобразование значений

```sql
-- Преобразование оценок в баллы
SELECT 
    s.student_id,
    x.student_name,
    c.course_title,
    c.grade,
    c.grade_points
FROM students_xml s,
XMLTable(
    '/student' 
    PASSING s.student_data
    COLUMNS
        student_name VARCHAR2(100) PATH 'name',
        courses      XMLType       PATH 'enrollments'
) x,
XMLTable(
    '/enrollments/course' 
    PASSING x.courses
    COLUMNS
        course_title VARCHAR2(100) PATH 'title',
        grade        VARCHAR2(2)   PATH 'grade',
        grade_points NUMBER        PATH '
            if (grade = "A") then 4
            else if (grade = "B") then 3
            else if (grade = "C") then 2
            else 1'
) c;
```

6. Агрегации и группировки в XQuery

Агрегации внутри XML

```sql
-- Вычисление статистики внутри XQuery
SELECT 
    student_id,
    XMLQuery(
        'let $courses := /student/enrollments/course
         return 
            <stats>
                <total_courses>{count($courses)}</total_courses>
                <avg_credits>{avg($courses/credits)}</avg_credits>
                <max_grade>{max($courses/grade)}</max_grade>
            </stats>' 
        PASSING student_data 
        RETURNING CONTENT
    ).getStringVal() as statistics
FROM students_xml;
```

Комбинирование SQL и XQuery агрегаций

```sql
-- SQL агрегации поверх XQuery преобразований
SELECT 
    faculty,
    COUNT(*) as student_count,
    AVG(course_count) as avg_courses_per_student,
    SUM(CASE WHEN has_excellent_grade = 1 THEN 1 ELSE 0 END) as excellent_students
FROM (
    SELECT 
        x.faculty,
        XMLQuery('count(/student/enrollments/course)' 
                PASSING s.student_data RETURNING CONTENT).getNumberVal() as course_count,
        CASE WHEN XMLEXISTS('/student[enrollments/course/grade="A"]' PASSING s.student_data)
             THEN 1 ELSE 0 END as has_excellent_grade
    FROM students_xml s,
    XMLTable(
        '/student' 
        PASSING s.student_data
        COLUMNS
            faculty VARCHAR2(50) PATH '@faculty'
    ) x
)
GROUP BY faculty;
```

7. Работа с пространствами имен (Namespaces)

Запросы с XML Namespaces

```sql
-- Работа с пространствами имен
SELECT 
    student_id,
    XMLQuery(
        'declare namespace uni = "http://university.edu/schema";
         /uni:student/uni:name/text()' 
        PASSING student_data 
        RETURNING CONTENT
    ).getStringVal() as student_name
FROM students_xml
WHERE XMLEXISTS(
    'declare namespace uni = "http://university.edu/schema";
     /uni:student[uni:status = "ACTIVE"]' 
    PASSING student_data
);
```

8. Сложные преобразования данных

Многоуровневое разворачивание XML

```sql
-- Сложная иерархия: Факультет → Студенты → Курсы → Оценки
SELECT 
    f.faculty_name,
    s.student_name,
    c.course_code,
    c.grade,
    a.assessment_date
FROM faculties_xml fac,
XMLTable(
    '/faculty' 
    PASSING fac.faculty_data
    COLUMNS
        faculty_name VARCHAR2(100) PATH 'name',
        students     XMLType       PATH 'students'
) f,
XMLTable(
    '/students/student' 
    PASSING f.students
    COLUMNS
        student_name VARCHAR2(100) PATH 'name',
        courses      XMLType       PATH 'enrollments'
) s,
XMLTable(
    '/enrollments/course' 
    PASSING s.courses
    COLUMNS
        course_code  VARCHAR2(20) PATH 'courseCode',
        grade        VARCHAR2(2)  PATH 'grade',
        assessments  XMLType      PATH 'assessments'
) c,
XMLTable(
    '/assessments/assessment' 
    PASSING c.assessments
    COLUMNS
        assessment_date DATE PATH '@date'
) a
WHERE c.grade = 'A';
```

9. Модификация XML данных в запросах

Обновление XML с преобразованием

```sql
-- Создание модифицированного XML
SELECT 
    student_id,
    XMLQuery(
        'copy $student := /student
         modify (
             for $course in $student/enrollments/course
             where $course/grade = "A" 
             return replace value of node $course/grade with "EXCELLENT"
         )
         return $student' 
        PASSING student_data 
        RETURNING CONTENT
    ) as modified_data
FROM students_xml;
```

10. Практический пример: Аналитический отчет

```sql
-- Комплексный аналитический отчет
WITH student_stats AS (
    SELECT 
        s.student_id,
        x.faculty,
        x.student_name,
        x.email,
        c.course_count,
        c.avg_grade_points,
        c.excellent_courses
    FROM students_xml s,
    XMLTable(
        '/student' 
        PASSING s.student_data
        COLUMNS
            faculty      VARCHAR2(50)  PATH '@faculty',
            student_name VARCHAR2(100) PATH 'name',
            email        VARCHAR2(100) PATH 'email'
    ) x,
    XMLTable(
        'let $courses := /student/enrollments/course
         return 
            <stats>
                <course_count>{count($courses)}</course_count>
                <avg_grade_points>{avg(
                    for $g in $courses/grade
                    return 
                        if ($g = "A") then 4
                        else if ($g = "B") then 3
                        else if ($g = "C") then 2
                        else 1
                )}</avg_grade_points>
                <excellent_courses>{count($courses[grade="A"])}</excellent_courses>
            </stats>' 
        PASSING s.student_data
        COLUMNS
            course_count      NUMBER PATH 'course_count',
            avg_grade_points NUMBER PATH 'avg_grade_points',
            excellent_courses NUMBER PATH 'excellent_courses'
    ) c
)
SELECT 
    faculty,
    COUNT(*) as total_students,
    ROUND(AVG(course_count), 2) as avg_courses,
    ROUND(AVG(avg_grade_points), 2) as avg_grade_points,
    SUM(excellent_courses) as total_excellent_grades
FROM student_stats
GROUP BY faculty
ORDER BY avg_grade_points DESC;
```

Ключевые преимущества XQuery в Oracle XML DB:

1. Мощные преобразования - сложные XML → реляционные данные
2. Гибкость - условная логика, циклы, агрегации
3. Производительность - выполняется на стороне СУБД
4. Стандартизация - основано на стандартах W3C
5. Интеграция - полная интеграция с SQL

Начните с простых EXTRACTVALUE/EXISTSNODE, затем переходите к XMLQUERY/XMLEXISTS, и для сложных преобразований используйте XMLTABLE.


## Пример 1: XQuery в SQL
Разберем этот запрос построчно с подробными комментариями:

```sql
-- Использование XQuery в SQL
SELECT 
    -- 1. Выбираем обычный реляционный столбец student_id из таблицы
    student_id,
    
    -- 2. XMLQuery() - функция для выполнения XQuery выражений над XMLType данными
    XMLQuery(
        -- 3. XPath выражение: выбираем текстовое содержимое элемента <name> 
        --    внутри элемента <student>
        '/student/name/text()' 
        
        -- 4. PASSING - связываем XMLType столбец student_data с XQuery выражением
        --    Теперь в XQuery можно обращаться к XML документу как к переменной
        PASSING student_data 
        
        -- 5. RETURNING CONTENT - указывает, что результат должен возвращаться 
        --    как XML фрагмент (XMLType), а не как скалярное значение
        RETURNING CONTENT
        
    -- 6. .getStringVal() - метод XMLType, который преобразует XML результат 
    --    в строку VARCHAR2. Если бы не вызвали этот метод, результат был бы XMLType
    ).getStringVal() as student_name,
    
    -- 7. Второй XMLQuery для подсчета количества курсов
    XMLQuery(
        -- 8. XPath функция count() подсчитывает количество элементов <course>
        --    внутри пути /student/enrollments/course
        'count(/student/enrollments/course)' 
        
        -- 9. Снова связываем со столбцом student_data
        PASSING student_data 
        
        -- 10. Возвращаем результат как XML фрагмент
        RETURNING CONTENT
        
    -- 11. .getNumberVal() - преобразует XML результат в число Oracle NUMBER
    --     Функция count() возвращает числовое значение, поэтому используем getNumberVal()
    ).getNumberVal() as course_count

-- 12. Указываем таблицу-источник данных
FROM students_xml

-- 13. WHERE с XMLEXISTS - фильтрация строк на основе XPath условия
WHERE XMLEXISTS(
    -- 14. XPath выражение в квадратных скобках - это предикат (условие фильтрации)
    --     Выбираем только тех студентов, у которых есть хотя бы один курс с оценкой "A"
    --     Путь: /student[условие] - найдет элементы <student>, удовлетворяющие условию
    '/student[enrollments/course/grade = "A"]' 
    
    -- 15. PASSING - связываем XML данные для проверки условия
    PASSING student_data
);
```

Детальное объяснение ключевых моментов:

Как работает XPath в этом запросе:

```xpath
/student/name/text()
```

· / - корень документа
· student - элемент первого уровня
· name - дочерний элемент student
· text() - функция, возвращающая текстовое содержимое элемента

```xpath
count(/student/enrollments/course)
```

· count() - XPath функция подсчета
· /student/enrollments/course - путь ко всем элементам course

```xpath
/student[enrollments/course/grade = "A"]
```

· [ ] - предикат (условие фильтрации)
· enrollments/course/grade = "A" - условие: значение элемента grade равно "A"

Пример данных, которые обрабатывает этот запрос:

```xml
<!-- Строка, которая БУДЕТ выбрана (есть grade="A") -->
<student>
    <name>John Doe</name>
    <enrollments>
        <course>
            <grade>A</grade>  <!-- Есть оценка A - условие выполняется -->
        </course>
        <course>
            <grade>B</grade>
        </course>
    </enrollments>
</student>

<!-- Строка, которая НЕ БУДЕТ выбрана (нет grade="A") -->
<student>
    <name>Jane Smith</name>
    <enrollments>
        <course>
            <grade>B</grade>  <!-- Нет оценки A - условие не выполняется -->
        </course>
    </enrollments>
</student>
```

Результат выполнения запроса:

STUDENT_ID STUDENT_NAME COURSE_COUNT
123 John Doe 2
124 Mike Brown 1

Альтернативные варианты записи:

```sql
-- Эквивалентная запись с EXTRACTVALUE (устаревшая)
SELECT 
    student_id,
    EXTRACTVALUE(student_data, '/student/name') as student_name,
    EXTRACT(student_data, 'count(/student/enrollments/course)').getNumberVal() as course_count
FROM students_xml
WHERE EXISTSNODE(student_data, '/student[enrollments/course/grade = "A"]') = 1;

-- Современная запись с XMLTABLE (рекомендуется)
SELECT 
    s.student_id,
    x.student_name,
    x.course_count
FROM students_xml s,
XMLTable(
    '/student' PASSING s.student_data
    COLUMNS
        student_name VARCHAR2(100) PATH 'name',
        course_count NUMBER PATH 'count(enrollments/course)'
) x
WHERE XMLEXISTS('/student[enrollments/course/grade = "A"]' PASSING s.student_data);
```

Этот запрос демонстрирует мощь интеграции XQuery и SQL в Oracle XML DB, позволяя выполнять сложные XML-ориентированные операции прямо в SQL запросах.


## Пример 2: XMLTABLE
Разберем этот мощный запрос построчно с подробными комментариями:

```sql
-- Преобразование XML в реляционный формат
SELECT 
    -- 1. s.student_id - обычный реляционный столбец из таблицы students_xml
    s.student_id,
    
    -- 2. x.name - столбец, извлеченный из XML функцией XMLTable
    --    Содержит значение элемента <name> из XML документа
    x.name,
    
    -- 3. x.email - значение элемента <email> из XML документа
    x.email,
    
    -- 4. c.course_code - значение атрибута/элемента courseCode из вложенного XML
    --    Этот столбец создается вторым вызовом XMLTable
    c.course_code,
    
    -- 5. c.grade - значение элемента <grade> из вложенного XML
    c.grade

-- 6. FROM students_xml s - основная таблица с XML данными
--    Псевдоним 's' для удобства обращения
FROM students_xml s,

-- 7. Первый вызов XMLTable - преобразует основной XML документ в реляционные столбцы
XMLTable(
    -- 8. XPath выражение '/student' - указывает корневой элемент для обработки
    --    XMLTable будет обрабатывать каждый элемент <student> в XML документе
    '/student' 
    
    -- 9. PASSING s.student_data - связывает столбец student_data с XPath выражением
    --    Для каждой строки таблицы s, student_data передается в XMLTable
    PASSING s.student_data
    
    -- 10. COLUMNS - определяет какие данные извлекать из XML и в какие столбцы
    COLUMNS
        -- 11. name VARCHAR2(100) PATH 'name' - создает столбец name типа VARCHAR2(100)
        --     Значение берется из элемента <name> внутри текущего <student>
        name VARCHAR2(100) PATH 'name',
        
        -- 12. email VARCHAR2(100) PATH 'email' - создает столбец email
        --     Значение из элемента <email> внутри <student>
        email VARCHAR2(100) PATH 'email',
        
        -- 13. courses XMLType PATH 'enrollments' - особый столбец!
        --     Сохраняет весь фрагмент <enrollments> как XMLType для дальнейшей обработки
        --     Это позволяет делать вложенную обработку XML
        courses XMLType PATH 'enrollments'
) x,  -- 14. Псевдоним 'x' для результата первого XMLTable

-- 15. Второй вызов XMLTable - обрабатывает вложенный XML из столбца courses
XMLTable(
    -- 16. XPath '/enrollments/course' - обрабатывает каждый элемент <course>
    --     внутри фрагмента <enrollments>, который сохранили в предыдущем шаге
    '/enrollments/course' 
    
    -- 17. PASSING x.courses - передаем XML фрагмент из столбца courses
    --     который создали в первом XMLTable
    PASSING x.courses
    
    -- 18. COLUMNS - определяем столбцы для данных о курсах
    COLUMNS
        -- 19. course_code VARCHAR2(20) PATH 'courseCode' - код курса
        --     Может быть из элемента <courseCode> или атрибута courseCode
        course_code VARCHAR2(20) PATH 'courseCode',
        
        -- 20. grade VARCHAR2(2) PATH 'grade' - оценка по курсу
        grade VARCHAR2(2) PATH 'grade'
) c  -- 21. Псевдоним 'c' для результата второго XMLTable

-- 22. WHERE c.grade = 'A' - фильтрация результатов
--     Оставляем только те строки, где оценка равна 'A'
WHERE c.grade = 'A';
```

Как работает процесс поэтапно:

Исходные данные в таблице students_xml:

```xml
<!-- student_id = 1 -->
<student>
    <name>John Doe</name>
    <email>john@university.edu</email>
    <enrollments>
        <course>
            <courseCode>MATH101</courseCode>
            <grade>A</grade>
        </course>
        <course>
            <courseCode>CS101</courseCode>
            <grade>B</grade>
        </course>
    </enrollments>
</student>

<!-- student_id = 2 -->
<student>
    <name>Jane Smith</name>
    <email>jane@university.edu</email>
    <enrollments>
        <course>
            <courseCode>PHYS101</courseCode>
            <grade>A</grade>
        </course>
    </enrollments>
</student>
```

Шаг 1: Первый XMLTable создает промежуточный результат:

student_id name email courses (XMLType)
1 John Doe john@university.edu <enrollments><course>...
2 Jane Smith jane@university.edu <enrollments><course>...

Шаг 2: Второй XMLTable разворачивает вложенные курсы:

Для student_id = 1 (2 курса):

· Курс 1: MATH101, grade A
· Курс 2: CS101, grade B

Для student_id = 2 (1 курс):

· Курс 1: PHYS101, grade A

Шаг 3: Декартово произведение + фильтрация:

Промежуточный результат до WHERE:

student_id name email course_code grade
1 John Doe john@university.edu MATH101 A
1 John Doe john@university.edu CS101 B
2 Jane Smith jane@university.edu PHYS101 A

Шаг 4: Применение WHERE c.grade = 'A':

Финальный результат:

student_id name email course_code grade
1 John Doe john@university.edu MATH101 A
2 Jane Smith jane@university.edu PHYS101 A

Ключевые особенности этого подхода:

1. Неявный JOIN между XMLTable:

· Первый XMLTable создает одну строку на каждого студента
· Второй XMLTable создает одну строку на каждый курс студента
· Результат - декартово произведение (все комбинации студент-курс)

2. Сохраняем контекст:

Даже после разворачивания вложенных курсов, мы сохраняем связь с исходными данными студента (name, email)

3. Эффективность:

Оба XMLTable выполняются параллельно, Oracle оптимизирует такой запрос

Альтернативная визуализация логики:

```sql
-- Эквивалентная логика с явными JOIN (псевдокод):
SELECT 
    s.student_id,
    student.name,
    student.email, 
    course.course_code,
    course.grade
FROM students_xml s
JOIN LATERAL ( -- Первый XMLTable
    SELECT 
        extract(name) as name,
        extract(email) as email,
        extract(enrollments) as courses
    FROM XMLTable('/student' PASSING s.student_data)
) student ON TRUE
JOIN LATERAL ( -- Второй XMLTable  
    SELECT
        extract(courseCode) as course_code,
        extract(grade) as grade
    FROM XMLTable('/enrollments/course' PASSING student.courses)
) course ON TRUE
WHERE course.grade = 'A';
```

Этот запрос демонстрирует мощь Oracle XML DB для преобразования сложных иерархических XML данных в привычный реляционный формат, сохраняя при этом все связи между данными.


## Динамическое преобразование XML в табличный вид без знания структуры
Вот несколько способов динамического преобразования XML в табличный вид без знания структуры:

1. Динамическое определение структуры через USER_TYPE_ATTRS

```sql
-- Сначала анализируем структуру XMLType столбца
SELECT 
    ut.type_name,
    ua.attr_name,
    ua.attr_type_name,
    ua.length,
    ua.precision
FROM user_types ut
JOIN user_type_attrs ua ON ut.type_name = ua.type_name
WHERE ut.type_name LIKE '%STUDENT%'
ORDER BY ut.type_name, ua.attr_no;
```

2. Универсальный разбор XML через XMLSequence

```sql
-- Разбор любого XML на элементы и атрибуты
SELECT 
    s.student_id,
    x.element_name,
    x.element_value,
    x.attribute_name,
    x.attribute_value
FROM students_xml s,
XMLTable(
    'for $i in //* 
     return <element name="{local-name($i)}" value="{$i/text()}">{
        for $attr in $i/@*
        return <attribute name="{local-name($attr)}" value="{$attr}"/>
     }</element>' 
    PASSING s.student_data
    COLUMNS
        element_name    VARCHAR2(100) PATH '@name',
        element_value   VARCHAR2(4000) PATH '@value',
        attributes      XMLType        PATH '.'
) x,
XMLTable(
    '/element/attribute' 
    PASSING x.attributes
    COLUMNS
        attribute_name  VARCHAR2(100) PATH '@name',
        attribute_value VARCHAR2(4000) PATH '@value'
) attr
WHERE s.student_id = 123;
```

3. Динамический PIVOT с определением столбцов

```sql
-- Сначала получаем все возможные имена элементов
WITH all_elements AS (
    SELECT DISTINCT
        EXTRACTVALUE(column_value, '/*/local-name()') as element_name
    FROM students_xml s,
    TABLE(XMLSequence(EXTRACT(s.student_data, '//*')))
    WHERE EXTRACTVALUE(column_value, '/*/local-name()') IS NOT NULL
),
element_values AS (
    SELECT 
        s.student_id,
        EXTRACTVALUE(column_value, '/*/local-name()') as element_name,
        EXTRACTVALUE(column_value, '/*/text()') as element_value
    FROM students_xml s,
    TABLE(XMLSequence(EXTRACT(s.student_data, '//*')))
    WHERE EXTRACTVALUE(column_value, '/*/text()') IS NOT NULL
)
SELECT *
FROM element_values
PIVOT (
    MAX(element_value)
    FOR element_name IN (
        SELECT element_name FROM all_elements WHERE ROWNUM <= 10
    )
);
```

4. Рекурсивный разбор XML иерархии

```sql
-- Рекурсивный CTE для разбора любой XML структуры
WITH xml_structure (student_id, path, element_name, element_value, lvl) AS (
    -- Базовый случай: корневые элементы
    SELECT 
        s.student_id,
        '/' || EXTRACTVALUE(column_value, '/*/local-name()') as path,
        EXTRACTVALUE(column_value, '/*/local-name()') as element_name,
        EXTRACTVALUE(column_value, '/*/text()') as element_value,
        1 as lvl
    FROM students_xml s,
    TABLE(XMLSequence(EXTRACT(s.student_data, '/*')))
    
    UNION ALL
    
    -- Рекурсивный случай: дочерние элементы
    SELECT 
        xs.student_id,
        xs.path || '/' || EXTRACTVALUE(column_value, '/*/local-name()'),
        EXTRACTVALUE(column_value, '/*/local-name()'),
        EXTRACTVALUE(column_value, '/*/text()'),
        xs.lvl + 1
    FROM xml_structure xs,
    students_xml s,
    TABLE(XMLSequence(EXTRACT(s.student_data, xs.path || '/*')))
    WHERE s.student_id = xs.student_id
      AND xs.lvl < 5  -- Ограничение глубины для производительности
)
SELECT 
    student_id,
    path,
    element_name,
    element_value,
    lvl
FROM xml_structure
WHERE element_value IS NOT NULL
ORDER BY student_id, path;
```

5. Универсальный парсинг через DBMS_XMLDOM

```sql
DECLARE
    CURSOR c_students IS 
        SELECT student_id, student_data FROM students_xml WHERE ROWNUM <= 5;
    
    v_doc DBMS_XMLDOM.DOMDocument;
    v_root DBMS_XMLDOM.DOMElement;
    v_node_list DBMS_XMLDOM.DOMNodeList;
    v_node DBMS_XMLDOM.DOMNode;
    v_name VARCHAR2(100);
    v_value VARCHAR2(4000);
BEGIN
    DBMS_OUTPUT.PUT_LINE('STUDENT_ID|ELEMENT_PATH|VALUE');
    DBMS_OUTPUT.PUT_LINE('----------|------------|-----');
    
    FOR rec IN c_students LOOP
        v_doc := DBMS_XMLDOM.newDOMDocument(rec.student_data);
        v_root := DBMS_XMLDOM.getDocumentElement(v_doc);
        
        -- Рекурсивный обход всех элементов
        print_xml_elements(v_root, '', rec.student_id);
        
        DBMS_XMLDOM.freeDocument(v_doc);
    END LOOP;
END;
/

-- Вспомогательная процедура для рекурсивного обхода
CREATE OR REPLACE PROCEDURE print_xml_elements(
    p_element DBMS_XMLDOM.DOMElement,
    p_path VARCHAR2,
    p_student_id NUMBER
) IS
    v_children DBMS_XMLDOM.DOMNodeList;
    v_child DBMS_XMLDOM.DOMNode;
    v_name VARCHAR2(100);
    v_value VARCHAR2(4000);
BEGIN
    v_name := DBMS_XMLDOM.getTagName(p_element);
    v_value := DBMS_XMLDOM.getNodeValue(DBMS_XMLDOM.getFirstChild(p_element));
    
    IF v_value IS NOT NULL THEN
        DBMS_OUTPUT.PUT_LINE(p_student_id || '|' || p_path || '/' || v_name || '|' || v_value);
    END IF;
    
    -- Рекурсивно обрабатываем дочерние элементы
    v_children := DBMS_XMLDOM.getChildNodes(p_element);
    FOR i IN 0..DBMS_XMLDOM.getLength(v_children)-1 LOOP
        v_child := DBMS_XMLDOM.item(v_children, i);
        IF DBMS_XMLDOM.getNodeType(v_child) = DBMS_XMLDOM.ELEMENT_NODE THEN
            print_xml_elements(
                DBMS_XMLDOM.makeElement(v_child),
                p_path || '/' || v_name,
                p_student_id
            );
        END IF;
    END LOOP;
END;
/
```

6. Практическое решение: Универсальный XML-парсер

```sql
-- Создаем функцию для динамического парсинга XML
CREATE OR REPLACE FUNCTION parse_xml_to_table(
    p_xml XMLTYPE
) RETURN SYS.ODCIVARCHAR2LIST PIPELINED IS
    
    v_doc DBMS_XMLDOM.DOMDocument;
    v_root DBMS_XMLDOM.DOMElement;
    
    PROCEDURE process_element(
        p_element DBMS_XMLDOM.DOMElement,
        p_path VARCHAR2
    ) IS
        v_children DBMS_XMLDOM.DOMNodeList;
        v_child DBMS_XMLDOM.DOMNode;
        v_name VARCHAR2(100);
        v_value VARCHAR2(4000);
    BEGIN
        v_name := DBMS_XMLDOM.getTagName(p_element);
        v_value := DBMS_XMLDOM.getNodeValue(DBMS_XMLDOM.getFirstChild(p_element));
        
        IF v_value IS NOT NULL THEN
            PIPE ROW(p_path || '/' || v_name || '=' || v_value);
        END IF;
        
        -- Обработка атрибутов
        DECLARE
            v_attrs DBMS_XMLDOM.DOMNamedNodeMap;
            v_attr DBMS_XMLDOM.DOMNode;
        BEGIN
            v_attrs := DBMS_XMLDOM.getAttributes(p_element);
            FOR i IN 0..DBMS_XMLDOM.getLength(v_attrs)-1 LOOP
                v_attr := DBMS_XMLDOM.item(v_attrs, i);
                PIPE ROW(p_path || '/' || v_name || '@' || 
                        DBMS_XMLDOM.getNodeName(v_attr) || '=' ||
                        DBMS_XMLDOM.getNodeValue(v_attr));
            END LOOP;
        END;
        
        -- Рекурсивная обработка детей
        v_children := DBMS_XMLDOM.getChildNodes(p_element);
        FOR i IN 0..DBMS_XMLDOM.getLength(v_children)-1 LOOP
            v_child := DBMS_XMLDOM.item(v_children, i);
            IF DBMS_XMLDOM.getNodeType(v_child) = DBMS_XMLDOM.ELEMENT_NODE THEN
                process_element(
                    DBMS_XMLDOM.makeElement(v_child),
                    p_path || '/' || v_name
                );
            END IF;
        END LOOP;
    END process_element;
    
BEGIN
    v_doc := DBMS_XMLDOM.newDOMDocument(p_xml);
    v_root := DBMS_XMLDOM.getDocumentElement(v_doc);
    
    process_element(v_root, '');
    
    DBMS_XMLDOM.freeDocument(v_doc);
    RETURN;
END;
/

-- Использование функции
SELECT 
    s.student_id,
    t.column_value as xml_path_value
FROM students_xml s,
TABLE(parse_xml_to_table(s.student_data)) t
WHERE ROWNUM <= 100;
```

7. Самый простой способ - через XMLTable с динамическими путями

```sql
-- Универсальный разбор без знания структуры
SELECT 
    s.student_id,
    elem.element_name,
    elem.element_value,
    attr.attribute_name, 
    attr.attribute_value
FROM students_xml s,
XMLTable(
    'for $e in //*
     return <row>
        <element_name>{local-name($e)}</element_name>
        <element_value>{string($e)}</element_value>
        <attributes>{
            for $a in $e/@*
            return <attribute name="{local-name($a)}" value="{$a}"/>
        }</attributes>
     </row>' 
    PASSING s.student_data
    COLUMNS
        element_name  VARCHAR2(100) PATH 'element_name',
        element_value VARCHAR2(4000) PATH 'element_value',
        attributes    XMLType        PATH 'attributes'
) elem,
XMLTable(
    '/row/attributes/attribute' 
    PASSING elem.attributes
    COLUMNS
        attribute_name  VARCHAR2(100) PATH '@name',
        attribute_value VARCHAR2(4000) PATH '@value'
) attr
WHERE elem.element_value IS NOT NULL
   OR attr.attribute_value IS NOT NULL;
```

Результат всех методов будет выглядеть так:

STUDENT_ID PATH VALUE
123 /student/name John Doe
123 /student/email john@edu.com
123 /student/@id 123
123 /student/courses/course/@code MATH101

Рекомендации:

1. Для разового анализа - используйте метод #7 (XMLTable с динамическими путями)
2. Для регулярного использования - создайте функцию как в методе #6
3. Для глубокого анализа структуры - используйте рекурсивный CTE (#4)
4. Для максимальной производительности - ограничьте глубину обхода

Эти методы позволяют исследовать любую XML структуру без предварительного знания ее схемы!
